# TP1 : Web scraping et APIs

## 1. SPARQL

### Imports

In [34]:
from datetime import datetime as dt
from SPARQLWrapper import SPARQLWrapper, JSON

### Obtenir la liste des bourgmestres de Bruxelles

In [36]:
endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"
sparql = SPARQLWrapper(endpoint)

# P27 = Country of citizenship 
# Q31 = Belgium
# P39 = Position held 
# Q33126365 = Mayor of Brussels
# P569 = date of birth
# P570 = date of death

statement = """
SELECT DISTINCT ?person ?personLabel ?dateBirth ?dateDeath WHERE {
    ?person wdt:P27 wd:Q31 .
    ?person wdt:P39 wd:Q33126365 .
    ?person wdt:P569 ?dateBirth .
    OPTIONAL {?person wdt:P570 ?dateDeath .}
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
}
ORDER BY ?dateBirth
"""

sparql.setQuery(statement)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

rows = results['results']['bindings']
print(f"\n{len(rows)} Mayor of Brussels\n")
print (results)


26 Mayor of Brussels

{'head': {'vars': ['person', 'personLabel', 'dateBirth', 'dateDeath']}, 'results': {'bindings': [{'person': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2306891'}, 'personLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Jan-Baptist Verlooy'}, 'dateBirth': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1746-12-22T00:00:00Z'}, 'dateDeath': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1797-05-04T00:00:00Z'}}, {'person': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q105605007'}, 'personLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Henri Joseph Van Langhenhoven'}, 'dateBirth': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1749-01-01T00:00:00Z'}}, {'person': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q16661300'}, 'personLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Louis Devos'}, 'da

### Filtrer pour ne conserver que les bourgmestres ayant vécus entre 1847 et 1978

In [37]:
#J'ai filré par date de naissance parce que je n'ai pas réussi à trouver comment filter 
#par leur date de service en tant que bourgmestre
#J'aurais aimé ne filtrer que les bourgmestres ayant été en service entre 1847 et 1978.

statement = """
SELECT DISTINCT ?person ?personLabel ?dateBirth ?dateDeath WHERE {
    ?person wdt:P27 wd:Q31 .
    ?person wdt:P39 wd:Q33126365 .
    ?person wdt:P569 ?dateBirth .
    OPTIONAL {?person wdt:P570 ?dateDeath .}
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
FILTER ((YEAR(?dateBirth) >=1846) && (YEAR(?dateBirth) <= 1978))
}
ORDER BY ?dateBirth
"""

sparql.setQuery(statement)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

rows = results['results']['bindings']
print(f"\n{len(rows)} Mayor of Brussels born between 1847 and 1978\n")
print (results)


12 Mayor of Brussels born between 1847 and 1978

{'head': {'vars': ['person', 'personLabel', 'dateBirth', 'dateDeath']}, 'results': {'bindings': [{'person': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2535128'}, 'personLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Adolphe Max'}, 'dateBirth': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1869-12-30T00:00:00Z'}, 'dateDeath': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1939-11-06T00:00:00Z'}}, {'person': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2589184'}, 'personLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Joseph Van De Meulebroeck'}, 'dateBirth': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1876-11-17T00:00:00Z'}, 'dateDeath': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1958-12-14T00:00:00Z'}}, {'person': {'type': 'uri'

## 2. API

### Imports

In [3]:
import requests

### Utiliser GeoDB Cities pour avoir des détails sur la ville de Bruxelles ou les villes aux alentours

http://geodb-cities-api.wirefreethought.com/

In [4]:
#API utilisé pour avoir des détails sur la ville de Bruxelles

url = "https://wft-geo-db.p.rapidapi.com/v1/geo/cities/Q239"

headers = {
    'x-rapidapi-host': "wft-geo-db.p.rapidapi.com",
    'x-rapidapi-key': "73ab91bba7msha57c8a85717c885p15fe59jsnbf7071066e7a"
    }

response = requests.request("GET", url, headers=headers)

print(response.text)

{"data":{"id":7918,"wikiDataId":"Q239","type":"CITY","city":"City of Brussels","name":"City of Brussels","country":"Belgium","countryCode":"BE","region":"Brussels-Capital Region","regionCode":"BRU","elevationMeters":null,"latitude":50.85045,"longitude":4.34878,"population":185103,"timezone":"Europe__Brussels","distance":null,"deleted":false}}


In [18]:
#API pour avoir des informations sur les villes autour de Bruxelles
url = "https://wft-geo-db.p.rapidapi.com/v1/geo/cities/Q239/nearbyCities"

querystring = {"radius":"100"}

headers = {
    'x-rapidapi-host': "wft-geo-db.p.rapidapi.com",
    'x-rapidapi-key': "73ab91bba7msha57c8a85717c885p15fe59jsnbf7071066e7a"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"data":[{"id":3453277,"wikiDataId":"Q240","type":"CITY","city":"Brussels Capital Region","name":"Brussels Capital Region","country":"Belgium","countryCode":"BE","region":"Brussels-Capital Region","regionCode":"BRU","latitude":50.85,"longitude":4.35,"population":1218255,"distance":0.06},{"id":3189062,"wikiDataId":"Q90870","type":"ADM2","city":"Arrondissement of Brussels-Capital","name":"Arrondissement of Brussels-Capital","country":"Belgium","countryCode":"BE","region":"Brussels-Capital Region","regionCode":"BRU","latitude":50.85,"longitude":4.35,"population":1208542,"distance":0.06},{"id":3091651,"wikiDataId":"Q242393","type":"CITY","city":"Woluwe-Saint-Pierre - Sint-Pieters-Woluwe","name":"Woluwe-Saint-Pierre - Sint-Pieters-Woluwe","country":"Belgium","countryCode":"BE","region":"Brussels-Capital Region","regionCode":"BRU","latitude":50.85505,"longitude":4.37067,"population":42119,"distance":1.01},{"id":3000152,"wikiDataId":"Q219244","type":"CITY","city":"Koekelberg","name":"Koekelbe